In [ ]:
'''
Runs a Tornado webserver on port 8000
Provides a simple UI showing configuration fields
Showing SSID, wpapsk, and certificate name and expiration dates

'''
import os
import time
import tornado.web
import OpenSSL
import datetime
import requests
from threading import Thread
import cs

import nest_asyncio
nest_asyncio.apply()

class StatusHandler(tornado.web.RequestHandler):
    def get(self):
        try:
            primary_device = cp.get('status/wan/primary_device')
            selapn = cp.get(f'status/wan/devices/{primary_device}/diagnostics/SELAPN')
            cfgapn = cp.get(f'status/wan/devices/{primary_device}/diagnostics/CFGAPN{selapn}')
            response_text = f'Current APN: {cfgapn}<br><br>'
            ssid = cp.get('config/wlan/radio/0/bss/0/ssid')
            wpapsk = cp.decrypt('config/wlan/radio/0/bss/0/wpapsk')
            response_text += f'WiFi SSID: {ssid}<br>Password:  {wpapsk}<br><br>'
            ports = cp.get('config/ethernet')
            for port in ports:
                response_text += f'Ethernet Port {port["port"]} enabled: {port["enabled"]}<br>'
            certs = cp.get('config/certmgmt/certs')
            for cert in certs:
                x509 = OpenSSL.crypto.load_certificate(OpenSSL.crypto.FILETYPE_PEM, cert["x509"])
                response_text += f'<br>Certificate Name: {cert["name"]}<br>Expires: {datetime.datetime.strptime(x509.get_notAfter().decode("ascii"), "%Y%m%d%H%M%SZ")}<br>'
            self.write(response_text)
        except Exception as e:
            print(e)


if __name__ == "__main__":
    cp = cs.CSClient('webserver')
    print('Starting...')
    application = tornado.web.Application([
        (r"/status", StatusHandler),
        (r"/(.*)", tornado.web.StaticFileHandler,
         {"path": os.getcwd(), "default_filename": "index.html"}),
    ])
    application.listen(8000)
    tornado.ioloop.IOLoop.instance().start()